In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim


import os
import sys
cur_dir = os.path.dirname(os.path.abspath("__file__"))  # Gets the current notebook directory
src_dir = os.path.join(cur_dir, '../')  # Constructs the path to the 'src' directory
# Add the 'src' directory to sys.path
if src_dir not in sys.path:
    sys.path.append(src_dir)

from src.constant import *
from tqdm.notebook import tqdm

from src.MyDataset import MyDataset

In [4]:
lookback = 20
dir = '../data/PandasData/Original/'

ds = MyDataset(lookback=lookback)

In [10]:
# df = pd.read_pickle('../data/PandasData/Original/PID001_NSL.pkl')
# f_per_sec = df.groupby('TimestampID').count().mean().mean()
# target_freq = 10
# resample_ratio = int(f_per_sec/target_freq)
# df = df.iloc[::resample_ratio, :]

In [5]:
def process_data(df_dir : str, target_freq : int = 10):
    df = pd.read_pickle(df_dir)
    f_per_sec = df.groupby('TimestampID').count().mean().mean()
    resample_ratio = int(f_per_sec/target_freq)
    df = df.iloc[::resample_ratio, :]
    # for origin
    df = df.drop(columns=['Confidence', 'Timestamp', 'TimestampID', 'DatapointID', 'PID', 'SCN', 'U_X', 'U_Y', 'U_Z', 'AGV_Z', 'User_Z', 'GazeOrigin_Z', 'User_Pitch', 'User_Yaw', 'User_Roll', 'EyeTarget'])
    return df

for file in os.listdir(dir):
    if file.endswith('.pkl'):
        df = pd.read_pickle(dir+file)
        ds.read_data(df)

train, test = ds.split_data(frac=0.8, shuffle=True, batch_size=4)
feature_dim = ds.feature_dim

/home/shaoze/Documents/Boeing/Boeing-Trajectory-Prediction/notebook/../src/MyDataset.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(X), torch.tensor(y)


In [11]:
# for origin
# df = df.drop(columns=['Confidence', 'Timestamp', 'TimestampID', 'DatapointID', 'PID', 'SCN', 'U_X', 'U_Y', 'U_Z', 'AGV_Z', 'User_Z', 'GazeOrigin_Z', 'User_Pitch', 'User_Yaw', 'User_Roll', 'EyeTarget'])


# numeric_df = df.select_dtypes(include=[np.number])
# numeric_df.drop(columns=['Timestamp', 'start_station_X', 'start_station_Y', 'end_station_X', 'end_station_Y',
#        'distance_from_start_station_X', 'distance_from_start_station_Y',
#        'distance_from_end_station_X', 'distance_from_end_station_Y','AGV distance X', 'AGV distance Y', ], inplace=True)

In [6]:
for i, (X, y) in enumerate(train):
    print(X.shape, y.shape)
    break

print(len(train), len(test))

torch.Size([4, 20, 26]) torch.Size([4, 20, 26])
207289 51823


In [7]:
class TraPredModel(nn.Module):
    def __init__(self, input_size = None, lookback = None):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=50, num_layers=lookback, batch_first=True)
        self.linear = nn.Linear(50, 2)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [8]:
model = TraPredModel(input_size=feature_dim, lookback=lookback)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# train_ds = DataLoader(TensorDataset(X_train, y_train), shuffle=True, batch_size=8)
# test_ds = DataLoader(TensorDataset(X_test, y_test), shuffle=False, batch_size=8)


In [13]:
n_epochs = 1
eval_step = 100000
# model = TraPredModel(input_size=numeric_df.shape[1], lookback=lookback)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")
model.to(device)

train_all = len(train)

for epoch in range(n_epochs):
    model.train()
    for step, (X_batch, y_batch) in tqdm(enumerate(train), total = train_all):
        X_batch = X_batch.float().to(device)
        y_batch = y_batch.float().to(device)

        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch[:, :, -2:])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation
        if (epoch * train_all + step + 1) % eval_step == 0:
            print(f"Start testing")
            with torch.no_grad():
                model.eval()
                all_test = len(test)
                test_rmse_all = []
                for X_test_batch, y_test_batch in tqdm(test):
                    X_test_batch = X_test_batch.float().to(device)
                    y_test_batch = y_test_batch.float().to(device)
                    y_pred = model(X_test_batch)
                    test_rmse = loss_fn(y_pred, y_test_batch[:, :, -2:])
                    if not torch.isnan(test_rmse):
                        test_rmse_all.append(test_rmse.item())

                print("Epoch %d: test RMSE %.4f" % (epoch, sum(test_rmse_all)/all_test))
            
            model.train()
        # breaku
 

Using cuda


  0%|          | 0/207289 [00:00<?, ?it/s]

Start testing


  0%|          | 0/51823 [00:00<?, ?it/s]

Epoch 0: test RMSE 0.0200
Start testing


  0%|          | 0/51823 [00:00<?, ?it/s]

Epoch 0: test RMSE 0.0205
